In [1]:
#IMPORT useful libraries to create and mofidy pandas frames.
import pandas as pd
import numpy as np
import re

In [2]:
#read the csv files into pandas frame
#D1_raw = pd.read_csv('2020_US_weekly_symptoms_dataset1.csv')
D1_raw = pd.read_csv('d1_try.csv')
D2_raw = pd.read_csv('aggregated_cc_by1.csv')

/Users/Reid/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#keep only US region-states in dataset 2.
D2_raw = D2_raw[D2_raw['open_covid_region_code'].str.contains("US-")]

In [4]:
#replace all zeros with np.nan to ease filtering.
D1_raw = D1_raw.replace(0, pd.np.nan)

#first remove all columns with more than 35% nan entires.
limitPerC = len(D1_raw) * 0.35
D1_raw = D1_raw.dropna(thresh=limitPerC,axis=1)

#second, remove all rows with more than 10% nan entries too
limitPerR = len(D1_raw.columns) * 0.1
D1_raw = D1_raw.dropna(thresh=limitPerR,axis=0)
D1_raw.shape

<ipython-input-4-5e0b0171e473>:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  D1_raw = D1_raw.replace(0, pd.np.nan)


(2040, 425)

In [5]:
#divide all columns containing symptoms by the mean of that feature to be able to compare with other regions.
feature_names = D1_raw.filter(regex='symptom:*', axis=1).columns                                                                                                                                           
D1_raw[feature_names] = D1_raw.groupby('open_covid_region_code').transform(lambda x: x / x.mean())[feature_names]

In [6]:
#resample all the data from d2 to weekly and region (might no be necessary but wtv.)
D2_raw['date'] = D2_raw['date'].astype('datetime64[ns]')
D2_raw = D2_raw.groupby('open_covid_region_code').resample('W-Mon', label='right', closed = 'right', on='date',how='sum').sum().reset_index().sort_values(by='open_covid_region_code')

In [7]:
#sort the whole d1-set to weekly and by region instead of daily (sum everything).
D1_raw['date'] = D1_raw['date'].astype('datetime64[ns]')
D1_raw = D1_raw.groupby('open_covid_region_code').resample('W-Mon', label='right', closed = 'right', on='date',how='sum').sum().reset_index().sort_values(by='open_covid_region_code')

In [8]:
#replace all zeros with np.nan to ease filtering.
D2_raw = D2_raw.replace(0, pd.np.nan)

#first remove all columns with more than 35% nan entires.
limitPerC = len(D1_raw) * .35
D2_raw = D2_raw.dropna(thresh=limitPerC,axis=1)

#second, remove all rows with more than 10% nan entries too
limitPerR = len(D2_raw.columns) * 0.1
D2_raw = D2_raw.dropna(thresh=limitPerR,axis=0)

<ipython-input-8-077484001461>:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  D2_raw = D2_raw.replace(0, pd.np.nan)


In [9]:
#add the two datasets together and reindex
df = D1_raw.append(D2_raw, ignore_index=True)

#replace all the nan entries with 0.
df = df.replace(pd.np.nan,0.0)
df.shape

<ipython-input-9-9daea2764eb6>:5: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  df = df.replace(pd.np.nan,0.0)


(3846, 427)

In [10]:
#regroup based on region and date essentially adding all together
df = df.groupby('open_covid_region_code').resample('W-Mon', label='right', closed = 'right', on='date',how='sum').sum().reset_index().sort_values(by='open_covid_region_code')

In [11]:
#sort based on region code and date
df = df.sort_values(['open_covid_region_code', 'date'], ascending=[True, True])
df.shape

(2246, 427)

In [12]:
#replace the dataframes 0s to nan, easier to sort later...
df = df.replace(0, pd.np.nan)
#delete rows, then columns, with 5% values with nan, just to filter one last time.
limitPerR = len(df.columns) * 0.05
df = df.dropna(thresh=limitPerR,axis=0)
df = df.dropna(subset=['hospitalized_cumulative'])
df = df.replace(pd.np.nan,0.0)
df.shape

<ipython-input-12-f6596ce7509d>:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  df = df.replace(0, pd.np.nan)
<ipython-input-12-f6596ce7509d>:7: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  df = df.replace(pd.np.nan,0.0)


(993, 427)

In [13]:
#save to the clean data
df.to_csv(r'clean_data.csv')